Multitask Recommender with item-item Retrieval model. 

We will build a multi-task recommender with using both implicit (product clicks) and explicit signals (ratings). Ranking and Retrieval stages will be used. For this model we will use item-to-item Retrieval model.



## Multitask Model

### Imports

In [1]:
! pip install tensorflow



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install -q tensorflow-recommenders
! pip install -q --upgrade tensorflow-datasets
! pip install -q scann

     |████████████████████████████████| 89 kB 3.4 MB/s 
     |████████████████████████████████| 4.7 MB 4.5 MB/s 
     |████████████████████████████████| 10.4 MB 4.0 MB/s 
     |████████████████████████████████| 578.0 MB 15 kB/s 
     |████████████████████████████████| 438 kB 53.3 MB/s 
     |████████████████████████████████| 1.7 MB 68.1 MB/s 
     |████████████████████████████████| 5.9 MB 34.8 MB/s 


In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_recommenders as tfrs

# import interactive table 
from google.colab import data_table
data_table.enable_dataframe_formatter()

# set seed
tf.random.set_seed(42)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preparing Dataset

In [5]:
# load data subset 
gdrive_path = '/content/drive/MyDrive/ModelingData'
path = os.path.join(gdrive_path, "ratings")

ratings = tf.data.Dataset.load(path)

In [6]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    'product_title': x['data']['product_title'], 
    'customer_id': x['data']['customer_id'], 
    'star_rating': x['data']['star_rating']
})
products = ratings.map(lambda x: x['product_title'])

In [7]:
# train-test split
tf.random.set_seed(42)
shuffled = ratings.shuffle(92_096, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(92_096)
test = shuffled.skip(92_096).take(23_024)

In [8]:
# vocabulary to map raw feature values to embedding vectors
product_titles = products.batch(50_000)
customer_ids = ratings.batch(110_000).map(lambda x: x['customer_id'])

unique_product_titles = np.unique(np.concatenate(list(product_titles)))
unique_customer_ids = np.unique(np.concatenate(list(customer_ids)))

### Implementing the model

There are two critical parts to multi-task recommenders:

They optimize for two or more objectives, and so have two or more losses.
They share variables between the tasks, allowing for transfer learning.
In this tutorial, we will define our models as before, but instead of having a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

In [9]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_customer_ids, mask_token=None),
  # We add 1 to account for the unknown token.
  tf.keras.layers.Embedding(len(unique_customer_ids) + 1, embedding_dimension)
])

product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_product_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_product_titles) + 1, embedding_dimension)
])

We will have two tasks: Ranking and Retrieval. 

In [ ]:
tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

In [ ]:
tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=products.batch(128)
    )
)

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [10]:
# define a model
class AmazonModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and product models.
    self.product_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_product_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_product_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_product_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_product_titles) + 1, embedding_dimension)
    ])

    # A small model to take in user and product embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=products.batch(128).map(self.product_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["product_title"])
    # And pick out the product features and pass them into the product model.
    product_embeddings = self.product_model(features["product_title"])

    return (
        user_embeddings,
        product_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and product embeddings.
        self.rating_model(
            tf.concat([user_embeddings, product_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("star_rating")

    user_embeddings, product_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, product_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

#### Rating-specialized model

We will start with model that only considers ratings. 

In [ ]:
rating_model = AmazonModel(rating_weight=1.0, retrieval_weight=0.0)
rating_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [13]:
# shuffle, batch, and cache train and test data
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
rating_model.fit(cached_train, epochs=3)
metrics = rating_model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
12/12 [==============================] - 1274s 105s/step - root_mean_squared_error: 2.4213 - factorized_top_k/top_1_categorical_accuracy: 1.0858e-05 - factorized_top_k/top_5_categorical_accuracy: 6.5149e-05 - factorized_top_k/top_10_categorical_accuracy: 7.6008e-05 - factorized_top_k/top_50_categorical_accuracy: 4.9948e-04 - factorized_top_k/top_100_categorical_accuracy: 9.3381e-04 - loss: 5.2712 - regularization_loss: 0.0000e+00 - total_loss: 5.2712
Epoch 2/3
12/12 [==============================] - 1284s 107s/step - root_mean_squared_error: 1.2111 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 6.5149e-05 - factorized_top_k/top_10_categorical_accuracy: 7.6008e-05 - factorized_top_k/top_50_categorical_accuracy: 5.1034e-04 - factorized_top_k/top_100_categorical_accuracy: 9.3381e-04 - loss: 1.4649 - regularization_loss: 0.0000e+00 - total_loss: 1.4649
Epoch 3/3
6/6 [==============================] - 342s 55s/step - root_

#### Retrieval-specialized model

In [ ]:
retrieval_model = AmazonModel(rating_weight=0.0, retrieval_weight=1.0)
retrieval_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
retrieval_model.fit(cached_train, epochs=3)
metrics = retrieval_model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

#### Joint model

Assigning equal weights to both tasks. 

In [11]:
joint_model = AmazonModel(rating_weight=1.0, retrieval_weight=1.0)
joint_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [14]:
joint_model.fit(cached_train, epochs=3)
metrics = joint_model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
12/12 [==============================] - 1301s 107s/step - root_mean_squared_error: 2.7123 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 2.1716e-05 - factorized_top_k/top_50_categorical_accuracy: 0.0544 - factorized_top_k/top_100_categorical_accuracy: 0.0958 - loss: 61713.2886 - regularization_loss: 0.0000e+00 - total_loss: 61713.2886
Epoch 2/3
12/12 [==============================] - 1365s 114s/step - root_mean_squared_error: 1.2147 - factorized_top_k/top_1_categorical_accuracy: 0.0122 - factorized_top_k/top_5_categorical_accuracy: 0.0139 - factorized_top_k/top_10_categorical_accuracy: 0.0188 - factorized_top_k/top_50_categorical_accuracy: 0.2136 - factorized_top_k/top_100_categorical_accuracy: 0.2628 - loss: 51142.6668 - regularization_loss: 0.0000e+00 - total_loss: 51142.6668
Epoch 3/3
6/6 [==============================] - 357s 58s/step - root_mean_squared

Accuracy rate at 38.8% for Top-100 recs on test data. 

### Serving and saving the model

In [15]:
# making predictions
trained_product_embeddings, trained_customer_embeddings, predicted_rating = joint_model({
      "customer_id": np.array(["52228204"]),
      "product_title": np.array(["Vader Bicycle Cycling Bike Road Offroad MTB Mountain Saddle Seat"])
  })
print("Predicted rating:")
print(predicted_rating)

Predicted rating:
tf.Tensor([[4.7740483]], shape=(1, 1), dtype=float32)


In [16]:
# recommending Top-10 products for customer 52228204

# Create a joint_model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(joint_model.product_model)
# recommends products out of the entire products dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((products.batch(100), products.batch(100).map(joint_model.product_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["52228204"]))
print(f"Recommendations for user 52228204: {titles[0, :10]}")

Recommendations for user 52228204: [b'Coleman Converta Cot' b'Coleman Converta Cot' b'Coleman Converta Cot'
 b'Coleman Converta Cot' b'Coleman Converta Cot' b'Coleman Converta Cot'
 b'Coleman Converta Cot' b'Coleman Converta Cot' b'Coleman Converta Cot'
 b'Coleman Converta Cot']


In [17]:
# model serving: saving the model to G-Drive

# Export the query model.
gdrive_path = '/content/drive/MyDrive/Models'
path = os.path.join(gdrive_path, "joint_model")

# Save the index.
tf.saved_model.save(index, path)

# Load it back; can also be done in TensorFlow Serving.
joint_model_2 = tf.saved_model.load(path)

# Pass a user id in, get top predicted movie titles back.
scores, titles = joint_model_2(["52228204"])

print(f"Recommendations: {titles[0][:3]}")


Recommendations: [b'Coleman Converta Cot' b'Coleman Converta Cot' b'Coleman Converta Cot']


In [18]:
# adding ScaNN layer
scann_index = tfrs.layers.factorized_top_k.ScaNN(joint_model.product_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((products.batch(100), products.batch(100).map(joint_model.product_model)))
)

In [19]:
# Get recommendations.
_, titles = scann_index(tf.constant(["52228204"]))
print(f"Recommendations for user 52228204: {titles[0, :10]}")

Recommendations for user 52228204: [b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)'
 b'TravelJohn-Disposable Urinal (6 pack)']


In [20]:
# exporting ScaNN layer

# Export the query model.
gdrive_path = '/content/drive/MyDrive/Models'
path = os.path.join(gdrive_path, "joint_model")

# Save the index.
tf.saved_model.save(
    index,
    path,
    options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
)

# Load it back; can also be done in TensorFlow Serving.
joint_model_2 = tf.saved_model.load(path)

# Pass a user id in, get top predicted movie titles back.
scores, titles = joint_model_2(["52228204"])

print(f"Recommendations: {titles[0][:10]}")

Recommendations: [b'Coleman Converta Cot' b'Coleman Converta Cot' b'Coleman Converta Cot'
 b'Coleman Converta Cot' b'Coleman Converta Cot' b'Coleman Converta Cot'
 b'Coleman Converta Cot' b'Coleman Converta Cot' b'Coleman Converta Cot'
 b'Coleman Converta Cot']


Accuracy rate at 38.8% for Top-100 recs on test data. Recommendations have a lot of repeats. 